In [2]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import sounddevice as sd
from scipy.io.wavfile import write
import pandas as pd

In [ ]:
def load_audio_files(directory, duration=20, sr=22050):
    labels = []
    spectrograms = []
    for filename in os.listdir(directory):
        if filename.endswith(".mp3") or filename.endswith(".wav"):
            path = os.path.join(directory, filename)
            audio, _ = librosa.load(path, sr=sr, duration=duration)
            audio = librosa.util.fix_length(audio, size=sr*duration)
            S = librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=128)
            S_dB = librosa.power_to_db(S, ref=np.max)
            spectrograms.append(S_dB)
            labels.append(os.path.splitext(filename)[0])
    return np.array(spectrograms), np.array(labels)

In [ ]:
def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')  # Ensure softmax for multi-class classification
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',  # Correct loss function
                  metrics=['accuracy'])
    return model

In [7]:
def record_audio(filename='temp_recording.wav', duration=20, sr=22050):
    print("Press Enter to start recording for {} seconds...")
    input("Press Enter to start recording:")
    recording = sd.rec(int(duration * sr), samplerate=sr, channels=2)
    sd.wait()  # Wait until recording is finished
    write(filename, sr, recording)  # Save as WAV file
    print("Recording finished.")
    return filename

directory = "C:\\Users\\Plaksha\\Desktop\\Sem 6\\Deep Learning\\AALets do it viks\\Data"
spectrograms, labels = load_audio_files(directory)
labels, uniques = pd.factorize(labels)

# Normalize spectrograms
spectrograms = spectrograms / np.max(spectrograms)

# Prepare data for training
X_train, X_test, y_train, y_test = train_test_split(spectrograms[..., np.newaxis], labels, test_size=0.2, random_state=42)

# Build and train the CNN model
input_shape = X_train.shape[1:]
num_classes = len(np.unique(labels))
model = build_cnn_model(input_shape, num_classes)
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

def predict_top_songs(audio_path, top_k=3):
    audio, _ = librosa.load(audio_path, sr=22050, duration=20)
    audio = librosa.util.fix_length(audio, size=22050*20)
    S = librosa.feature.melspectrogram(y=audio, sr=22050, n_mels=128)
    S_dB = librosa.power_to_db(S, ref=np.max)
    S_dB = S_dB[np.newaxis, ..., np.newaxis] / np.max(S_dB)
    predictions = model.predict(S_dB)
    top_indices = np.argsort(predictions[0])[-top_k:][::-1]
    return [uniques[i] for i in top_indices]

# Record and predict
input_audio_path = record_audio()
top_songs = predict_top_songs(input_audio_path)
print("Top Predicted Songs:", top_songs)


Epoch 1/20
3/3 [==============================] - 18s 4s/step - loss: 178934176.0000 - accuracy: 0.0000e+00 - val_loss: 63380152.0000 - val_accuracy: 0.0000e+00
Epoch 2/20
3/3 [==============================] - 11s 4s/step - loss: 29180164.0000 - accuracy: 0.0125 - val_loss: 17378088.0000 - val_accuracy: 0.0000e+00
Epoch 3/20
3/3 [==============================] - 12s 4s/step - loss: 21306736.0000 - accuracy: 0.0125 - val_loss: 9786749.0000 - val_accuracy: 0.0000e+00
Epoch 4/20
3/3 [==============================] - 12s 4s/step - loss: 7586764.0000 - accuracy: 0.0125 - val_loss: 4590802.0000 - val_accuracy: 0.0000e+00
Epoch 5/20
3/3 [==============================] - 13s 4s/step - loss: 2113841.7500 - accuracy: 0.0250 - val_loss: 81422.3594 - val_accuracy: 0.0500
Epoch 6/20
3/3 [==============================] - 12s 4s/step - loss: 34956.2773 - accuracy: 0.0500 - val_loss: 6955.7046 - val_accuracy: 0.0000e+00
Epoch 7/20
3/3 [==============================] - 12s 4s/step - loss: 2002.50